对预处理的中英文文本进行统计分析，包括计算在收集样本上英语字母和单词或汉字的概率和熵，并利用收集的英文文本验证齐夫定律(Zipf’s law)，分析过程中设置25%、50%、75%和100%的文本量，观察不同文本量对结果的影响，同时在分析过程中绘制相应的图表进行展示。

In [ ]:
# 库导入
import numpy as np
import pandas as pd
import collections as ct
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
'''分别为中文和英文设计不同的文本处理函数，对于中文直接对汉字进行相关的统计分析，而对于英文则分别进行字母和单词的统计分析，此外还有用于计算熵的函数。'''

# 中文文本处理函数
def zh_process(text):
    character_counts = ct.Counter(text)
    character_counts.pop(' ', None)
    character_stats = pd.DataFrame(
        character_counts.items(), 
        columns=['character', 'count']
    )   # 构造一个DataFrame，包含汉字及其计数，df形式方便后续分析与作图
    total_characters = character_stats['count'].sum()   # 统计总汉字数
    character_stats['prob'] = character_stats['count'] / total_characters   # 计算每个汉字的概率
    return character_stats # 返回包含汉字、计数、概率和熵的DataFrame

# 英文文本处理函数
def en_process(text):
    text = text.lower() # 先统一转换为小写字母
    letter_counts = ct.Counter(c for c in text if c.isalpha())  # 统计字母
    word_counts = ct.Counter(text.split())  # 统计单词
    letter_stats = pd.DataFrame(
        letter_counts.items(),
        columns=['letter', 'count']
    )
    word_stats = pd.DataFrame(
        word_counts.items(),
        columns=['word', 'count']
    )   # 分别构造字母和单词的DataFrame
    total_letters = letter_stats['count'].sum()
    total_words = word_stats['count'].sum()
    letter_stats['prob'] = letter_stats['count'] / total_letters
    word_stats['prob'] = word_stats['count'] / total_words
    return letter_stats, word_stats 

# 熵函数  
def calculate_entropy(dataframe):
    entropy = sum(-p * np.log2(p) for p in dataframe['prob'] if p > 0)
    return entropy

中文语料分析

In [ ]:
zh_file_path = 'zh_wikipedia.txt'   # 设置文件路径
with open(zh_file_path, 'r', encoding='utf-8') as f:
    zh_text = f.read()

zh_stats = zh_process(zh_text)
zh_entropy = calculate_entropy(zh_stats)
print(zh_entropy)
zh_stats = zh_stats.sort_values(by='count', ascending=False).reset_index(drop=True)
zh_stats.head(20)

In [ ]:
# 绘制汉字概率分布图（前20个汉字）

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.family'] = 'sans-serif'

plt.figure(figsize=(12, 6))
sns.barplot(x='character', y='prob', data=zh_stats.head(20))
plt.title('Top 20 Chinese Characters by Probability')